In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from pyswarms.single import GlobalBestPSO
import matplotlib.pyplot as plt

# Step 1: Load Otto dataset
otto_df = pd.read_csv("train.csv")  # 请确保和本文件同目录，或改为完整路径
X = otto_df.drop(columns=["id", "target"])
y = LabelEncoder().fit_transform(otto_df["target"])

# Step 2: Standardize features
X = StandardScaler().fit_transform(X)

# Step 3: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: PSO Settings
activation_options = ['relu', 'tanh', 'logistic']
solver_options = ['adam', 'sgd']

n_particles = 10
dimensions = 5
max_iter = 30
np.random.seed(42)

# Search space: hidden_layer_size, activation_idx, learning_rate_init, alpha, solver_idx
bounds = (
    np.array([10, 0, 0.0001, 0.00001, 0]),
    np.array([100, 2, 0.1, 0.1, 2])
)
init_pos = np.random.uniform(low=bounds[0], high=bounds[1], size=(n_particles, dimensions))

# Step 5: Define PSO objective
def objective_function(swarm):
    losses = []
    for params in swarm:
        try:
            hidden = int(round(params[0]))
            act = activation_options[int(round(params[1]))]
            lr = float(params[2])
            alpha = float(params[3])
            solver = solver_options[int(round(params[4]))]

            model = MLPClassifier(
                hidden_layer_sizes=(hidden,),
                activation=act,
                learning_rate_init=lr,
                alpha=alpha,
                solver=solver,
                max_iter=300,
                random_state=42
            )
            score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
            losses.append(-score)
        except:
            losses.append(1.0)
    return np.array(losses)

# Step 6: Run PSO
options = {'c1': 1.5, 'c2': 1.5, 'w': 0.5}
optimizer = GlobalBestPSO(
    n_particles=n_particles,
    dimensions=dimensions,
    options=options,
    bounds=bounds,
    init_pos=init_pos
)
cost, best_pos = optimizer.optimize(objective_function, iters=max_iter)

# Step 7: Decode best parameters
best_hidden = int(round(best_pos[0]))
best_act = activation_options[int(round(best_pos[1]))]
best_lr = float(best_pos[2])
best_alpha = float(best_pos[3])
best_solver = solver_options[int(round(best_pos[4]))]

print("\n✅ Best Parameters Found:")
print(f"   hidden_layer_sizes = ({best_hidden},)")
print(f"   activation = '{best_act}'")
print(f"   learning_rate_init = {best_lr}")
print(f"   alpha = {best_alpha}")
print(f"   solver = '{best_solver}'")

# Step 8: Final Model Evaluation
final_model = MLPClassifier(
    hidden_layer_sizes=(best_hidden,),
    activation=best_act,
    learning_rate_init=best_lr,
    alpha=best_alpha,
    solver=best_solver,
    max_iter=300,
    random_state=42
)
final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_test)
final_acc = accuracy_score(y_test, y_pred)
print(f"🎯 Final Test Accuracy: {final_acc:.4f}")

# Step 9: Plot convergence
plt.figure(figsize=(10, 5))
plt.plot(-np.array(optimizer.cost_history), marker='o')
plt.title("PSO Optimization of MLP (Otto Dataset)")
plt.xlabel("Iteration")
plt.ylabel("Best Cross-Validation Accuracy")
plt.grid(True)
plt.tight_layout()
plt.show()
